<a href="https://colab.research.google.com/github/josedavidcortes-com/finalComputerVision/blob/main/SVM_David_Cortes_Final_Exam_Computer_Vision_CSC752_UT1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **SVM**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC

#metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import seaborn as sns
from sklearn import svm

import matplotlib.image as mpimg
import random

%matplotlib inline

import os
from skimage.transform import resize
from skimage.io import imread

**Training**

In [3]:
#lets create the array of categories
shoes_categories_trn=['adidas','converse','nike']

#load the information from "train" folder, (images) 
trn_input_data_arr  = [] #input array
trn_output_data_arr = [] #output array
trn_path_dataset    = '/content/drive/MyDrive/ColabNotebooks/datasets/ComputerVision/shoe_dataset/train/'

#path which contains all the categories of images
for i in shoes_categories_trn:
    print(f'loading the category: {i}')
    path = os.path.join(trn_path_dataset,i)
    for img in os.listdir(path):
        img_array = imread( os.path.join(path,img) )
        img_resized = resize( img_array , (150,150,3) )
        trn_input_data_arr.append( img_resized.flatten())
        trn_output_data_arr.append( shoes_categories_trn.index(i))

trn_flat_data = np.array( trn_input_data_arr )
trn_target = np.array( trn_output_data_arr )
#dataframe to store and operate the data
trn_df = pd.DataFrame( trn_flat_data ) 
trn_df['Target'] = trn_target
#input data
x_trn = trn_df.iloc[:,:-1]  
#output data
y_trn = trn_df.iloc[:,-1] 

loading the category: adidas
loading the category: converse
loading the category: nike


In [4]:
trn_df

,0,1,2,3,4,5,6,7,8,9,...,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target
0,0.560431,0.556510,0.540824,0.563412,0.559490,0.543804,0.560196,0.554902,0.539216,0.557255,...,0.770588,0.794118,0.794118,0.764706,0.796471,0.792549,0.761961,0.797255,0.793333,0
1,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,...,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0
2,0.921569,0.925490,0.937255,0.921569,0.925490,0.940784,0.921569,0.925490,0.940000,0.921569,...,0.921569,0.925490,0.941176,0.921569,0.925490,0.941176,0.921569,0.925490,0.941176,0
3,0.882353,0.894118,0.913725,0.882353,0.894118,0.913725,0.882353,0.894118,0.913725,0.882353,...,0.882353,0.894118,0.913725,0.882353,0.894118,0.913725,0.882353,0.894118,0.913725,0
4,0.697647,0.695176,0.659137,0.529725,0.518314,0.485255,0.252353,0.231373,0.203725,0.359647,...,0.959608,0.885686,0.769412,0.968627,0.886000,0.778824,0.969647,0.884549,0.788824,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,...,0.867843,0.871765,0.848235,0.867843,0.871765,0.848235,0.867020,0.870941,0.847412,2
707,0.485451,0.473333,0.454902,0.486275,0.470863,0.458706,0.474510,0.458824,0.447059,0.479098,...,0.407451,0.455098,0.490392,0.398549,0.449529,0.484824,0.395255,0.446235,0.481529,2
708,0.988235,0.988235,0.988235,0.988235,0.988235,0.988235,0.988235,0.988235,0.988235,0.988235,...,0.988235,0.988235,0.988235,0.988235,0.988235,0.988235,0.988235,0.988235,0.988235,2
709,0.700627,0.527490,0.196863,0.722784,0.728353,0.444588,0.717255,0.760980,0.504314,0.750980,...,0.835294,0.855490,0.839804,0.835294,0.856078,0.840392,0.835294,0.856078,0.840392,2


**Testing**

In [5]:
#load the information from "train", (images and categories)
shoes_categories = [ 'adidas' , 'converse' , 'nike' ]
input_data_arr = [] #input array
output_data_arr = [] #output array
path_dataset = '/content/drive/MyDrive/ColabNotebooks/datasets/ComputerVision/shoe_dataset/test/'

#path which contains all the categories of images
for i in shoes_categories:
    print(f'loading the category: {i}')
    path = os.path.join( path_dataset , i )
    for img in os.listdir( path ):
        img_array = imread( os.path.join(path,img ) )
        img_resized = resize( img_array , ( 150 , 150 , 3 ) )
        input_data_arr.append( img_resized.flatten() )
        output_data_arr.append( shoes_categories.index(i) )

flat_data = np.array(input_data_arr)
target = np.array(output_data_arr)
#dataframe
tst_df=pd.DataFrame(flat_data) 
tst_df['Target']=target
#input data 
x_tst=tst_df.iloc[:,:-1] 
#output data
y_tst=tst_df.iloc[:,-1] 

loading the category: adidas
loading the category: converse
loading the category: nike


In [6]:
tst_df

,0,1,2,3,4,5,6,7,8,9,...,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target
0,0.130784,0.130784,0.130784,0.126941,0.126941,0.126941,0.129412,0.129412,0.129412,0.142000,...,0.832745,0.832745,0.832745,0.840353,0.840353,0.840353,0.862431,0.862431,0.862431,0
1,0.876471,0.879569,0.856039,0.835294,0.831882,0.811882,0.762941,0.751765,0.720392,0.731373,...,0.937255,0.925490,0.917647,0.937255,0.925490,0.917647,0.937255,0.925490,0.917647,0
2,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,...,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0
3,0.917647,0.921569,0.937255,0.917647,0.921569,0.937255,0.917647,0.921569,0.937255,0.917647,...,0.917647,0.921569,0.937255,0.917647,0.921569,0.937255,0.917647,0.921569,0.937255,0
4,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,...,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,0.898039,0.898039,0.898039,0.898039,0.898039,0.898039,0.898039,0.898039,0.898039,0.898039,...,0.898039,0.898039,0.898039,0.898039,0.898039,0.898039,0.898039,0.898039,0.898039,2
110,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,...,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,2
111,0.956863,0.956863,0.956863,0.956863,0.956863,0.956863,0.956863,0.956863,0.956863,0.956863,...,0.956863,0.956863,0.956863,0.956863,0.956863,0.956863,0.956863,0.956863,0.956863,2
112,0.474510,0.482353,0.470588,0.474510,0.482353,0.470588,0.474510,0.482353,0.470588,0.474510,...,0.474510,0.482353,0.470588,0.474510,0.482353,0.470588,0.474510,0.482353,0.470588,2


In [7]:
#Lets prepare the variables to inyect on SVM functions(create the model) and for testing purpose.

#training data
x_train = x_trn
y_train = y_trn

#testing data
x_test  = x_tst
y_test  = y_tst

In [8]:
#setting the params for GridSearchCV
#https://towardsdatascience.com/gridsearchcv-for-beginners-db48a90114ee
#C: factor
#gamma: the gamma 
#kernel: rbf/poly... etc
params = { 'C' : [10] , 'gamma' : [0.0001] , 'kernel' : ['rbf'] }

In [9]:
svc = svm.SVC( probability = True )

GridSearchCV is a tool to fine tune the parameters of your model(in my case i know which is the best because I did spend 6 hours waiting for the best parameters)

In [10]:
#estimator: object being used
#param_grid: dictionary with all of the parameters to try
#scoring: evaluation metric
#cv: cross-validation, # of cv folds xeach combination of parameters
model = GridSearchCV( svc , param_grid=params )

In [11]:
model.fit( x_train , y_train )

GridSearchCV(estimator=SVC(probability=True),
             param_grid={'C': [10], 'gamma': [0.0001], 'kernel': ['rbf']})

In [12]:
#https://www.askpython.com/python/examples/python-predict-function
y_pred = model.predict( x_test )
print ( "Predict : ")
print ( y_pred )

Predict : 
[2 2 0 0 0 1 1 2 0 1 0 1 0 0 2 2 2 0 2 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0
 0 1 0 2 2 1 2 1 2 1 2 1 1 0 0 1 2 1 1 2 1 2 1 0 0 1 1 1 1 1 1 1 1 2 1 0 0
 1 1 2 2 1 0 2 1 2 1 0 2 2 2 1 0 2 1 2 1 0 2 0 2 2 2 0 0 1 2 2 0 2 1 0 1 0
 2 1 1]


In [13]:
print ( "Actual : " )
print ( np.array( y_test ) )

Actual : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2]


In [14]:
#get the accuracy score
print ( f"The model is { accuracy_score ( y_pred , y_test ) * 100 } % accurate" )

The model is 54.385964912280706 % accurate


In [15]:
#confusion matrix
print( "Confusion matrix \n", confusion_matrix( y_pred , y_test ) )

Confusion matrix for linear kernel 
 [[23  7 10]
 [ 8 22 11]
 [ 7  9 17]]


In [16]:
#calculate the precision, recall, and f1-score
print("Analysis of our SVM Model: \n" , classification_report( y_pred , y_test ) )

Analysis of our SVM Model: 
               precision    recall  f1-score   support

           0       0.61      0.57      0.59        40
           1       0.58      0.54      0.56        41
           2       0.45      0.52      0.48        33

    accuracy                           0.54       114
   macro avg       0.54      0.54      0.54       114
weighted avg       0.55      0.54      0.55       114

